In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import regex as re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [75]:
important_cols = ['agency_abbr', 'loan_type_name', 'property_type_name',
       'loan_purpose_name', 'owner_occupancy_name', 'loan_amount_000s',
       'preapproval_name', 'action_taken_name', 'state_abbr',
       'county_name', 'census_tract_number', 'applicant_ethnicity_name',
       'co_applicant_ethnicity_name', 'applicant_race_name_1',
       'co_applicant_race_name_1','applicant_sex_name',
       'co_applicant_sex_name', 'applicant_income_000s', 'purchaser_type_name',
       'denial_reason_name_1', 'hoepa_status_name', 'lien_status_name', 'population',
       'minority_population', 'hud_median_family_income',
       'tract_to_msamd_income', 'number_of_owner_occupied_units',
       'number_of_1_to_4_family_units']
df_2017 = pd.read_csv('data/hmda_2017_nationwide_all-records_labels.csv',usecols = important_cols)
df_2017['denied'] = df_2017[['denial_reason_name_1']].applymap(lambda x : 1 if x==x else 0)
df_2017.head()

,agency_abbr,loan_type_name,property_type_name,loan_purpose_name,owner_occupancy_name,loan_amount_000s,preapproval_name,action_taken_name,state_abbr,county_name,...,denial_reason_name_1,hoepa_status_name,lien_status_name,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,denied
0,HUD,Conventional,One-to-four family dwelling (other than manufa...,Refinancing,Owner-occupied as a principal dwelling,53.0,Not applicable,Application withdrawn by applicant,PA,Philadelphia County,...,NaN,Not a HOEPA loan,Secured by a first lien,3202.0,97.279999,57400.0,47.540001,710.0,1314.0,0
1,CFPB,Conventional,One-to-four family dwelling (other than manufa...,Refinancing,Owner-occupied as a principal dwelling,168.0,Not applicable,Application denied by financial institution,WA,Spokane County,...,Credit history,Not a HOEPA loan,Secured by a first lien,3733.0,4.580000,63900.0,86.239998,861.0,1241.0,1
2,HUD,Conventional,One-to-four family dwelling (other than manufa...,Refinancing,Owner-occupied as a principal dwelling,103.0,Not applicable,File closed for incompleteness,UT,Salt Lake County,...,NaN,Not a HOEPA loan,Secured by a first lien,5498.0,37.919998,75400.0,63.939999,1270.0,1658.0,0
3,HUD,Conventional,One-to-four family dwelling (other than manufa...,Refinancing,Not owner-occupied as a principal dwelling,88.0,Not applicable,Loan originated,MO,Greene County,...,NaN,Not a HOEPA loan,Secured by a first lien,3566.0,11.830000,55200.0,74.290001,573.0,1261.0,0
4,HUD,Conventional,One-to-four family dwelling (other than manufa...,Refinancing,Owner-occupied as a principal dwelling,90.0,Not applicable,Application withdrawn by applicant,IL,Cook County,...,NaN,Not a HOEPA loan,Secured by a first lien,2910.0,48.660000,77500.0,79.250000,599.0,26.0,0


In [ ]:
df_2017[df_2017['state_abbr'] == 'CA']['county_name'].unique()

array(['Riverside County', 'Alameda County', 'Sacramento County',
       'San Bernardino County', 'Fresno County', 'Contra Costa County',
       'Santa Cruz County', 'Los Angeles County', 'Butte County',
       'Solano County', 'San Diego County', 'San Mateo County',
       'Amador County', 'Orange County', 'Stanislaus County',
       'El Dorado County', 'Santa Clara County', 'Monterey County',
       'Calaveras County', 'Ventura County', 'Glenn County',
       'Madera County', 'Colusa County', 'Tehama County', 'Sutter County',
       'Del Norte County', 'Sonoma County', 'Tulare County',
       'Placer County', 'Humboldt County', 'Nevada County',
       'San Luis Obispo County', 'Merced County', 'San Benito County',
       'Alpine County', 'Kern County', 'Santa Barbara County',
       'San Joaquin County', 'San Francisco County', 'Kings County',
       'Imperial County', 'Shasta County', 'Marin County', 'Napa County',
       nan, 'Yolo County', 'Plumas County', 'Siskiyou County',
     

## Clean Data

In [101]:
sd_df = df_2017[df_2017['county_name'] == 'San Diego County']
columns_to_drop = ['county_name','state_abbr','denial_reason_name_1','hud_median_family_income']
cleaned_sd_df = sd_df.drop(columns = columns_to_drop).dropna(subset = ['applicant_income_000s','number_of_1_to_4_family_units','loan_amount_000s'])
print(cleaned_sd_df.shape)
encoded_sd_df = pd.get_dummies(cleaned_sd_df)
print(encoded_sd_df.shape)
encoded_sd_df.head()

(139107, 25)
(139107, 86)


,loan_amount_000s,census_tract_number,applicant_income_000s,population,minority_population,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,denied,agency_abbr_CFPB,...,"purchaser_type_name_Life insurance company, credit union, mortgage bank, or finance company",purchaser_type_name_Loan was not originated or was not sold in calendar year covered by register,purchaser_type_name_Other type of purchaser,purchaser_type_name_Private securitization,hoepa_status_name_HOEPA loan,hoepa_status_name_Not a HOEPA loan,lien_status_name_Not applicable,lien_status_name_Not secured by a lien,lien_status_name_Secured by a first lien,lien_status_name_Secured by a subordinate lien
908,432.0,30.03,84.0,5045.0,75.379997,83.919998,989.0,1608.0,0,0,...,0,0,0,0,0,1,0,0,1,0
1070,273.0,27.05,58.0,4863.0,88.650002,71.419998,834.0,1281.0,1,0,...,0,1,0,0,0,1,0,0,1,0
1184,215.0,31.03,72.0,6686.0,92.209999,80.080002,1184.0,1661.0,0,0,...,0,0,0,0,0,1,0,0,1,0
2786,501.0,29.02,75.0,6846.0,60.240002,86.570000,983.0,1717.0,0,1,...,0,1,0,0,0,1,0,0,1,0
3026,1210.0,1.00,491.0,2716.0,13.030000,222.570007,1114.0,1283.0,1,1,...,0,1,0,0,0,1,0,0,1,0


In [102]:
cleaned_sd_df.isnull().sum(axis = 0).sort_values()

agency_abbr                       0
number_of_owner_occupied_units    0
tract_to_msamd_income             0
minority_population               0
population                        0
lien_status_name                  0
hoepa_status_name                 0
purchaser_type_name               0
applicant_income_000s             0
co_applicant_sex_name             0
applicant_sex_name                0
number_of_1_to_4_family_units     0
co_applicant_race_name_1          0
co_applicant_ethnicity_name       0
applicant_ethnicity_name          0
census_tract_number               0
action_taken_name                 0
preapproval_name                  0
loan_amount_000s                  0
owner_occupancy_name              0
loan_purpose_name                 0
property_type_name                0
loan_type_name                    0
applicant_race_name_1             0
denied                            0
dtype: int64

In [11]:
def find_col(x):
    cols = list(df_2017.columns)
    for col in cols:
        pats = re.findall(col,x)
        if len(pats) > 0:
            return col
    return 0


In [35]:
sorted_denial_rates = df_2017.groupby('state_abbr').mean()[['denied']].sort_values(by = 'denied')
counts_df = df_2017.groupby('state_abbr').count()[['respondent_id']]
grouped_df = sorted_denial_rates.join(counts_df).rename(columns={'denied':'Denial Rate','respondent_id':'Count'})
grouped_df

,Denial Rate,Count
state_abbr,,
WY,0.000000,13
ND,0.000000,32
LA,0.025624,10420
SC,0.029556,21620
NE,0.031270,28750
VA,0.041273,29414
CT,0.044368,28376
PR,0.050000,20
DC,0.052632,19


In [44]:
cutoff1 = .11
cutoff2 = .16
group1 = grouped_df[grouped_df['Denial Rate'] < cutoff1]
count1 = group1['Count'].sum()
group2 = grouped_df[(grouped_df['Denial Rate'] >= cutoff1) & (grouped_df['Denial Rate'] < cutoff2)]
count2 = group2['Count'].sum()
group3 = grouped_df[grouped_df['Denial Rate'] >= cutoff2]
count3 = group3['Count'].sum()

print(count1,count2,count3)

396833 282200 319826


## Train Model

In [103]:
train = encoded_sd_df.drop(columns = ['denied'])
target = encoded_sd_df['denied']
x_train, x_test, y_train, y_test = train_test_split(train,target,test_size = .4)

In [112]:
model = LogisticRegression()
model.fit(x_train,y_train)
train_preds = model.predict(x_train)
test_preds = model.predict(x_test)

C:\Users\maxym\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [111]:
print(classification_report(y_train,train_preds,labels = [0,1]))
print(classification_report(y_test,test_preds,labels = [0,1]))

C:\Users\maxym\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.91      1.00      0.95     75566
           1       0.00      0.00      0.00      7898

    accuracy                           0.91     83464
   macro avg       0.45      0.50      0.48     83464
weighted avg       0.82      0.91      0.86     83464

              precision    recall  f1-score   support

           0       0.90      1.00      0.95     50266
           1       0.00      0.00      0.00      5377

    accuracy                           0.90     55643
   macro avg       0.45      0.50      0.47     55643
weighted avg       0.82      0.90      0.86     55643

